# there is the SHAPE dataset of 50 sequences, under 4 conditions... 

we want to see of our accessibility is good

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
import sys
sys.path.insert(0,'..')
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

import matplotlib.pyplot as plt
import simushape as ss 
from scipy.stats import spearmanr as spear
import pandas
import numpy as np


# Load data

In [ ]:
import rna_tools.rna_io as rio
import pandas

types = ["cellfree",
"incell",
"kasugamycin"]

def getdata(typ):
    return rio.get_all_data("../data/weeks194_orig/%s.react" % typ,"../data/weeks194_orig/%s.dbn" % typ)  # {key: rea, seq, stru}

data = [getdata(t) for t in types] # 2 has bad results :)




# FIG 2   -- "weeks 50" paper

In [ ]:
# GRAPH FROM PAPER 
def window(stuff):
    return [ np.median(np.array( [z for z in stuff[i:i+51] if z != None] )) for i in range(0, len(stuff)-51)  ]
papr = pandas.DataFrame( [ window(data[a]["rpsP_rimM_trmD_rplS_96"][0][:500]) for a in range(3)],index=types)
#papr.T.plot.line()
import seaborn as sns
sns.set_context("talk")
sns.set(style="whitegrid")
g=sns.lineplot(data=papr.T, dashes=False)
g.set_xlabel('Transcript')
g.set_ylabel('SHAPE reactivity')
plt.title("Experimental Reavtivity for mRNA#96")
g.set(ylim=(0, 1))
#fig = g.get_figure()
#fig.savefig("output.pdf")

#   

In [ ]:
# SHAKER RECONSTRUCTION   # GRADIENT BOOST OPTIMIZED
def window(stuff):
    return [ np.median(np.array( [z for z in stuff[i:i+51] if z != None] )) for i in range(0, len(stuff)-50)  ]

keys = data[0].keys()
keys.remove("rpsP_rimM_trmD_rplS_96")
keys = keys[:10]
models = [ss.make_model(dat,keys,model=ss.make_xgbreg()) for dat in data ]
predictions = [ss.predict(mod, data[0]["rpsP_rimM_trmD_rplS_96"][1][:500]) for mod in models]

mystuff = pandas.DataFrame( [ window(p) for p in predictions],index=types)
mystuff.T.plot.line()

In [ ]:
def nv(dat):
    arrays = np.hstack([a for a,b,c in dat.values()])
    return np.mean(arrays[arrays!=None])

normaliationvalues= map(nv,data)
windowed = [ [ep/z for ep in window(p)] for p,z in zip(predictions,normaliationvalues)]
mystuff = pandas.DataFrame(windowed,index=types)
mystuff.T.plot.line()

In [ ]:
# SHAKER RECONSTRUCTION 
def window(stuff):
    return np.array([ np.median(np.array( [z for z in stuff[i:i+51] if z != None] )) for i in range(0, len(stuff)-50)  ])

keys = data[0].keys()
keys.remove("rpsP_rimM_trmD_rplS_96")
keys = keys[:10]
models = [ss.make_model(dat,keys,model=ss.make_forestregressor()) for dat in data ]
predictions = [ss.predict(mod, data[0]["rpsP_rimM_trmD_rplS_96"][1]) for mod in models]
mystuff = pandas.DataFrame( [ window(p) for p in predictions],index=types)
mystuff.T.plot.line()

In [ ]:
# RANDOM FOREST OPTIMIZED
#predictions = [ss.predict(mod, data[0]["rpsP_rimM_trmD_rplS_96"][1])[:500] for mod in models] 

def nv(dat):
    arrays = np.hstack([a for a,b,c in dat.values()])
    return np.mean(arrays[arrays!=None])
normaliationvalues= map(nv,data)
windowed = [ window(p)/z for p,z in zip(predictions,normaliationvalues)]
mystuff = pandas.DataFrame(windowed,index=types)
mystuff.T.plot.line()

# FIG 6   -- "weeks 50" paper

In [ ]:


# SHAKER RECONSTRUCTION   # GRADIENT BOOST OPTIMIZED
def window(stuff):
    return [ np.median(np.array( [z for z in stuff[i:i+51] if z != None] )) for i in range(0, len(stuff)-50)  ]

keys = data[0].keys()
keys.remove("rpmB_rpmG_147")
keys = keys[:10]
models = [ss.make_model(dat,keys,model=ss.make_xgbreg()) for dat in data ]
predictions = [ss.predict(mod, data[0]["rpmB_rpmG_147"][1]) for mod in models]

# predictions 
mystuff = pandas.DataFrame( [ window(p) for p in predictions],index=types)
mystuff.T.plot.line()



In [ ]:
from rna_tools import rnaplfold
# accessibility based on predictions 
acc =  [ rnaplfold.rnaplfold(data[0]["rpmB_rpmG_147"][1], p) for p in predictions]  
mystuff = pandas.DataFrame(acc,index=types)
mystuff.T.plot.line()

# accessibility windowed
mystuff = pandas.DataFrame(map(window,acc),index=types)
mystuff.T.plot.line()



# ACCESS OROGINAL 
papr = pandas.DataFrame( [ window(   rnaplfold.rnaplfold(data[a]["rpmB_rpmG_147"][1],data[a]["rpmB_rpmG_147"][0])  ) for a in range(3)],index=types)
papr.T.plot.line()

---

In [ ]:
(papr - mystuff).T.plot.line()

# FIG 2B

In [ ]:
%%time
import data.weeks194_orig.remove_genes as d


# get data
keys = data[0].keys()

keys_predict = ["rpsP_rimM_trmD_rplS_96","dusB_fis_130"]

keys_train = [k for k in  keys if k not in keys_predict]


# make model
models =[ss.make_xgbreg()  for dat in data]
models = [ ss.make_model(dat,keys_train, model=mod)for dat,mod in zip(data,models) ]





In [ ]:
%%time
# PREDICTION
from rna_tools.rnaplfold import rnaplfold

import notebook_helper as nb
asd=nb.getgenedict()

def window(stuff):
    return [ np.median(np.array( [z for z in stuff[i:i+51] if z != None] )) for i in range(0, len(stuff)-51)  ]



# predict
def predwrap(k,mod):
    seq = data[0][k][1]
    res = ss.predict(mod, seq[:600] )
    return res

def plfoldrap(k):
    seq = data[0][k][1]
    res = rnaplfold(seq[:600])
    return res


k = keys_predict[0]
predictions = [predwrap(k,mod)  for mod in models]
predictions.append(plfoldrap(k))




In [ ]:
realdata = [data[i][k][0][:600] for i in range(3)]
realdata.append(plfoldrap(k))

In [ ]:
import copy
realdata2 = copy.deepcopy(realdata)
realdata2 = [window(p) for p in realdata2]

realdata2 = [np.hstack([[np.nan]*25,p]) for p in realdata2]

realdata2.append(nb.get_genetrack(k, data[0],asd)[:600])

mystuff = pandas.DataFrame(realdata2,index=types+["plfold","gene"])
mystuff.T.plot.line(figsize=(12,4))

In [ ]:
import copy
predictions2 = copy.deepcopy(predictions)
predictions2 = [window(p) for p in predictions2]

predictions2 = [np.hstack([[np.nan]*25,p]) for p in predictions2]

predictions2.append(nb.get_genetrack(k, data[0],asd)[:600])

mystuff = pandas.DataFrame(predictions2,index=types+["plfold","gene"])
mystuff.T.plot.line(figsize=(12,4))

# Fig 3d

In [ ]:
keys = data[0].keys()
keys_train = keys[:10]
keys_predict = keys[-25:]
models = [ss.make_model(dat,keys,model=ss.make_xgbreg()) for dat in data ]


predictions = [[ss.predict(mod, data[0][k][1][:150]) for k in keys_predict] for mod in models]


# predictions 
#mystuff = pandas.DataFrame( [ window(p) for p in predictions],index=types)
#mystuff.T.plot.line()


In [ ]:
res = [np.array(p).mean(axis=0) for p in predictions]

mystuff = pandas.DataFrame([ window(p) for p in res],index=types)
mystuff.T.plot.line()

# DO THE SAME FOR GENESTARTSITE +-100

In [ ]:
%%time
import data.weeks194_orig.remove_genes as d



# get data
keys = data[0].keys()
keys_train = keys[:20]
keys_predict = keys[-25:]


#class sklearn.linear_model.LogisticRegression(penalty=’l2’, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver=’warn’, max_iter=100, multi_class=’warn’, verbose=0, warm_start=False, n_jobs=None)[source]¶
#sklearn.linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)[source]


import sklearn.linear_model as lim

# make model
models = [ ss.make_model(dat,keys_train, model=ss.make_forestregressor())for dat in data ]


# test again with n_estimators = 20



In [ ]:
%%time
# PREDICTION
from rna_tools.rnaplfold import rnaplfold

def window(stuff):
    return [ np.median(np.array( [z for z in stuff[i:i+51] if z != None] )) for i in range(0, len(stuff)-50)  ]

keys_predict = keys[-25:]
#keys_predict = keys[-2:]

# select gene start 
genstartdict = d.read_genes()
def get_genestart_region(k,data):
    index = int(k.split("_")[-1])
    genestart = genstartdict[index][0][0]
    geneend = genstartdict[index][0][1]
    
    sequence = data[0][k][1][:geneend]
    endpointindex = min(genestart+150, geneend)
    
    pre = sequence[max(genestart-150,0):genestart]
    post =   sequence[genestart:min(endpointindex, len(sequence))]
    answer= pre+post , abs(min(genestart-150,0)), max(genestart+150-len(sequence),0) 
    if (len(answer[0])+answer[1]+answer[2])!= 300:
        print answer, k,sequence, geneend
    print ".",
    return answer
    
# predict
def predwrap(k,mod):
    seq, padstart, padend = get_genestart_region(k,data)
    res = ss.predict(mod, seq )
    a=np.full((padstart,), np.nan, dtype=float)
    b = np.full((padend,), np.nan, dtype=float)
    ret= np.hstack( [a,res, b] )
    return ret

def plfoldrap(k):
    seq, padstart, padend = get_genestart_region(k,data)
    res = rnaplfold ( seq )
    a=np.full((padstart,), np.nan, dtype=float)
    b = np.full((padend,), np.nan, dtype=float)
    ret= np.hstack( [a,res, b] )
    return ret

predictions = [[predwrap(k,mod) for k in keys_predict] for mod in models]
predictions.append([plfoldrap(k) for k in keys_predict])
# we want the mean at nucleodide possition [-150.. 150] over all sequences...
res = [np.nanmean(np.array(p),axis=0) for p in predictions]

# windowing to smoothe the plot
mystuff = pandas.DataFrame([ window(p) for p in res],index=types+["plfold"])
mystuff.T.plot.line(ylim=(0, 1.2))


In [ ]:
#Original DAta
def window(stuff):
    return [ np.median(np.array( [z for z in stuff[i:i+51] if z != None] )) for i in range(0, len(stuff)-50)  ]


# select gene start 
genstartdict = d.read_genes()
def get_genestart_region2(k,data,exp_type):
    index = int(k.split("_")[-1])
    genestart = genstartdict[index][0][0]
    geneend = genstartdict[index][0][1]
    
    sequence = data[exp_type][k][0][:geneend]
    endpointindex = min(genestart+150, geneend)
    
    pre = sequence[max(genestart-150,0):genestart]
    post =   sequence[genestart:min(endpointindex, len(sequence))]
    
    pre = [e if e != None else np.nan for e in pre ]
    post = [e if e != None else np.nan for e in post ]
    
    answer= pre+post , abs(min(genestart-150,0)), max(genestart+150-len(sequence),0) 
    if (len(answer[0])+answer[1]+answer[2])!= 300:
        print answer, k,sequence, geneend
    print ".",
    return answer

def plfoldrap(k):
    seq, padstart, padend = get_genestart_region(k,data) # this is the verison above
    res = rnaplfold ( seq )
    a=np.full((padstart,), np.nan, dtype=float)
    b = np.full((padend,), np.nan, dtype=float)
    ret= np.hstack( [a,res, b] )
    return ret

# predict
def predwrap(k, exp):
    seq, padstart, padend = get_genestart_region2(k,data,exp)
    res =  seq 
    a=np.full((padstart,), np.nan, dtype=float)
    b = np.full((padend,), np.nan, dtype=float)
    ret= np.hstack( [a,res, b] )
    return ret

predictions = [[predwrap(k,mod) for k in keys_predict] for mod in range(3)]
predictions.append([plfoldrap(k) for k in keys_predict])
# we want the mean at nucleodide possition [-150.. 150] over all sequences...
#res = [np.array(p).mean(axis=0) for p in predictions]

res = [np.nanmean(np.array(p),axis=0) for p in predictions]


# windowing to smoothe the plot
mystuff = pandas.DataFrame([ window(p) for p in res],index=types+["plfold"])
mystuff.T.plot.line(ylim=(0, 1.2))